In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv("global_student_migration.csv")


In [4]:
df.head()

,student_id,origin_country,destination_country,destination_city,university_name,course_name,field_of_study,year_of_enrollment,scholarship_received,enrollment_reason,graduation_year,placement_status,placement_country,placement_company,starting_salary_usd,gpa_or_score,visa_status,post_graduation_visa,language_proficiency_test,test_score
0,S00001,Finland,Russia,Moscow,Lomonosov Moscow State University,Computer Science,Engineering,2021,No,Higher Ranking,2024,Placed,Russia,Microsoft,36416,3.92,Tier 4,Work Permit,TOEFL,7.8
1,S00002,UK,Germany,Aachen,RWTH Aachen,Civil Engineering,Law,2023,Yes,Job Opportunities,2024,Placed,Germany,Google,32956,2.60,Study Permit,OPT,PTE,7.9
2,S00003,Ireland,Canada,Vancouver,University of British Columbia,Law,Arts,2019,No,Scholarship,2020,Not Placed,NaN,NaN,0,2.72,F1,Blue Card,NaN,0.0
3,S00004,UAE,UK,Birmingham,University of Birmingham,Data Science,Social Sciences,2021,Yes,Quality of Life,2023,Placed,UK,Apple,50892,3.71,Schengen Student Visa,PSW,NaN,0.0
4,S00005,South Africa,Germany,Stuttgart,University of Stuttgart,Business Administration,Law,2020,Yes,Political Stability,2021,Placed,Germany,IBM,54790,2.96,Schengen Student Visa,OPT,PTE,8.2


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   student_id                 5000 non-null   object 
 1   origin_country             5000 non-null   object 
 2   destination_country        5000 non-null   object 
 3   destination_city           5000 non-null   object 
 4   university_name            5000 non-null   object 
 5   course_name                5000 non-null   object 
 6   field_of_study             5000 non-null   object 
 7   year_of_enrollment         5000 non-null   int64  
 8   scholarship_received       5000 non-null   object 
 9   enrollment_reason          5000 non-null   object 
 10  graduation_year            5000 non-null   int64  
 11  placement_status           5000 non-null   object 
 12  placement_country          2509 non-null   object 
 13  placement_company          2509 non-null   objec

In [2]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px


df = pd.read_csv("global_student_migration.csv")

# Set up the Dash app
app = dash.Dash(__name__)
app.title = "Global Student Migration Interactive Dashboard"

# ----------------------------
#  Dashboard Layout
# ----------------------------
app.layout = html.Div([
    html.H1("🌍 Global Student Migration Dashboard", style={'textAlign': 'center'}),

    html.P(
        "This dashboard walks you through the journey of student migration — "
        "where students go, why they pick certain destinations, how they perform academically, "
        "what their career outcomes look like, and which courses are most in demand.",
        style={'textAlign': 'center', 'fontSize': 16}
    ),

    # ----------------------------
    #  Filters: Country & Year
    # ----------------------------
    html.Div([
        html.Label("Select Destination Country"),
        dcc.Dropdown(
            id='country_dropdown',
            options=[{'label': i, 'value': i} for i in sorted(df['destination_country'].dropna().unique())],
            value=sorted(df['destination_country'].dropna().unique())[0],
            clearable=False
        ),
        html.Br(),

        html.Label("Select Year of Enrollment"),
        dcc.Slider(
            id='year_slider',
            min=int(df['year_of_enrollment'].min()),
            max=int(df['year_of_enrollment'].max()),
            step=1,
            value=int(df['year_of_enrollment'].min()),
            marks={i: str(i) for i in range(int(df['year_of_enrollment'].min()), 
                                           int(df['year_of_enrollment'].max()) + 1)}
        ),
    ], style={'width': '50%', 'margin': 'auto'}),

    html.Br(),

    # ----------------------------
    # Row 1: Map + Reasons
    # ----------------------------
    html.Div([
        html.Div([
            html.H3("1️⃣ Selected Country Focus"),
            html.P("An interactive map zooming in on your selected destination, "
                   "showing how many students enrolled that year."),
            dcc.Graph(id='map_chart')
        ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top'}),

        html.Div([
            html.H3("2️⃣ Why Students Enroll"),
            html.P("A bar chart breaking down the top reasons students choose "
                   "this destination in the selected year."),
            dcc.Graph(id='reason_chart'),
            html.Div(id='reason_narrative', style={'fontStyle': 'italic', 'padding': '10px'})
        ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top'}),
    ]),

    # ----------------------------
    # 📊 Row 2: Salaries + GPA/Test Scores
    # ----------------------------
    html.Div([
        html.Div([
            html.H3("3️⃣ Salary Outcomes After Graduation"),
            html.P("A box plot comparing starting salaries for graduates placed "
                   "in different countries."),
            dcc.Graph(id='salary_chart')
        ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top'}),

        html.Div([
            html.H3("4️⃣ Academic & Language Proficiency"),
            html.P("A scatter plot showing how GPA relates to language test scores."),
            dcc.Graph(id='scatter_chart', style={'width': '98%'})
        ], style={'width': '48%', 'display': 'inline-block', 'verticalAlign': 'top'}),
    ]),

    # ----------------------------
    # 📚 Row 3: Course Popularity
    # ----------------------------
    html.Div([
        html.H3("5️⃣ Most Popular Courses Among Students"),
        html.P("A bar chart showing which courses students choose most often "
               "for your selected destination and year."),
        dcc.Graph(id='course_chart'),
        html.Div(id='course_narrative', style={'fontStyle': 'italic', 'padding': '10px'})
    ])
])

# ----------------------------
# 🔄 Callback: Updates all charts when user changes filters
# ----------------------------
@app.callback(
    Output('map_chart', 'figure'),
    Output('reason_chart', 'figure'),
    Output('salary_chart', 'figure'),
    Output('scatter_chart', 'figure'),
    Output('reason_narrative', 'children'),
    Output('course_chart', 'figure'),
    Output('course_narrative', 'children'),
    Input('country_dropdown', 'value'),
    Input('year_slider', 'value')
)
def update_charts(selected_country, selected_year):
    """
    Updates all dashboard visuals based on the selected country and year.
    """
    
    # Filter the dataset to match the user’s selection
    selected_df = df[(df['destination_country'] == selected_country) &
                     (df['year_of_enrollment'] == selected_year)]
    
    # ----------------------------
    # 🗺 MAP: Highlight selected country
    # ----------------------------
    if selected_df.empty:
        map_fig = px.choropleth(title="No data for this selection")
    else:
        count = len(selected_df)
        map_fig = px.choropleth(
            selected_df.groupby('destination_country').size().reset_index(name='count'),
            locations='destination_country',
            locationmode='country names',
            color='count',
            title=f"{selected_country} — {count} Students in {selected_year}",
            color_continuous_scale=px.colors.sequential.Plasma
        )
        map_fig.update_geos(fitbounds="locations", visible=False)

    # ----------------------------
    # 📌 Reasons for enrollment
    # ----------------------------
    if not selected_df.empty:
        reason_counts = selected_df['enrollment_reason'].value_counts().reset_index()
        reason_counts.columns = ['Reason', 'Count']
        reason_counts['Percent'] = (reason_counts['Count'] / reason_counts['Count'].sum() * 100).round(1)
        reason_counts = reason_counts.sort_values(by='Count', ascending=False)
        colors = ['#ff7f0e' if i == 0 else '#1f77b4' for i in range(len(reason_counts))]
        
        reason_fig = px.bar(
            reason_counts, x='Reason', y='Count',
            text=reason_counts.apply(lambda row: f"{row['Count']} ({row['Percent']}%)", axis=1),
            color=reason_counts.index,
            color_discrete_sequence=colors
        )
        reason_fig.update_traces(textposition="outside", showlegend=False)
        
        # Create a short narrative summary
        top_reason = reason_counts.iloc[0]
        reason_story = f"In {selected_year}, most students choosing {selected_country} cited '{top_reason['Reason']}' — {top_reason['Percent']}%."
    else:
        reason_fig = px.bar(title="No data")
        reason_story = "No data available."

    # ----------------------------
    # 💰 Salary outcomes
    # ----------------------------
    salary_df = selected_df[selected_df['starting_salary_usd'] > 0]
    salary_fig = px.box(
        salary_df,
        x='placement_country',
        y='starting_salary_usd',
        points="all",
        title=f"Starting Salary by Placement Country ({selected_country})"
    )

    # ----------------------------
    # 📈 GPA vs Language Scores
    # ----------------------------
    scatter_df = selected_df[selected_df['test_score'] > 0]
    scatter_fig = px.scatter(
        scatter_df,
        x='gpa_or_score',
        y='test_score',
        color='language_proficiency_test',
        size='gpa_or_score',
        title=f"GPA vs. Language Test Score ({selected_country})"
    )

    # ----------------------------
    # 📚 Course Popularity
    # ----------------------------
    if not selected_df.empty and 'course_name' in selected_df.columns:
        course_counts = selected_df['course_name'].value_counts().reset_index()
        course_counts.columns = ['Course', 'Count']
        course_counts['Percent'] = (course_counts['Count'] / course_counts['Count'].sum() * 100).round(1)
        course_counts = course_counts.sort_values(by='Count', ascending=False)
        colors = ['#2ca02c' if i == 0 else '#1f77b4' for i in range(len(course_counts))]
        
        course_fig = px.bar(
            course_counts,
            x='Course',
            y='Count',
            text=course_counts.apply(lambda row: f"{row['Count']} ({row['Percent']}%)", axis=1),
            color=course_counts.index,
            color_discrete_sequence=colors
        )
        course_fig.update_traces(textposition="outside", showlegend=False)
        
        top_course = course_counts.iloc[0]
        course_story = f"In {selected_year}, the most chosen course in {selected_country} was '{top_course['Course']}' — {top_course['Percent']}% of students."
    else:
        course_fig = px.bar(title="No course data")
        course_story = "No course data available."

    return map_fig, reason_fig, salary_fig, scatter_fig, reason_story, course_fig, course_story

# ----------------------------
# 🚀 Run the app
# ----------------------------
if __name__ == '__main__':
    app.run()
